In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import os

_current_fname = Path(locals()["__vsc_ipynb_file__"])
os.chdir(_current_fname.parent)

In [3]:
import multiprocess as mp

mp.set_start_method("spawn")


In [4]:
def run_in_separate_process(
    run_id,
    device=0,
    min_mask_area=2000,
):
    print("Using device: ", device)
    import os

    os.environ["CUDA_VISIBLE_DEVICES"] = str(device)
    from pathlib import Path

    import copick
    import joblib
    import my_sam2_utils as utils
    import numpy as np
    import skimage
    import torch

    # use bfloat16 for the entire notebook.
    torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

    if torch.cuda.get_device_properties(0).major >= 8:
        # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

    # select the device for computation
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print(f"using device: {device}")

    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

    from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
    from sam2.build_sam import build_sam2, build_sam2_tomogram_predictor

    copick_config = (
        "/hpc/projects/group.czii/krios1.processing/copick/24jul29c/run002/copick_config.json"
    )

    # Open Copick Project and Query All Available Runs
    root = copick.from_file(copick_config)
    run = root.get_run(run_id)
    vol = utils.get_tomogram(run)

    print(f"Preprocessing the volume for {run_id}")

    vol_normed = vol.copy()
    vol_normed = (vol - vol.min()) / (vol.max() - vol.min())
    vol_normed = vol_normed * 2 - 1

    zSlice = 90  # int(points.mean(axis=0)[0])
    deltaZ = 50  # int((np.max(points[:, 0]) - np.min(points[:, 0]))) * 2

    min_mask_area = 2000

    #
    image = utils.project_tomogram(vol_normed, zSlice, deltaZ)

    # Extend From Grayscale to RGB
    image = np.repeat(image[..., None], 3, axis=2)

    sam2_checkpoint = (
        "/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/checkpoints/sam2_hiera_large.pt"
    )
    model_cfg = "sam2_hiera_l.yaml"

    sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=False)
    # mask_generator = SAM2AutomaticMaskGenerator(sam2)
    mask_generator = SAM2AutomaticMaskGenerator(
        model=sam2,
        points_per_side=32,
        points_per_batch=64,
        pred_iou_thresh=0.7,
        stability_score_thresh=0.92,
        stability_score_offset=0.7,
        crop_n_layers=1,
        box_nms_thresh=0.7,
        crop_n_points_downscale_factor=2,
        min_mask_region_area=2000,
        use_m2m=True,
    )

    print("Running automatic segmentation for projection")
    masks = mask_generator.generate(image)

    # Filter masks based on the area key
    masks = [mask for mask in masks if mask["area"] >= min_mask_area]

    print("Saving teh masks")
    out_dir = Path("cryosam_outputs")
    out_dir.mkdir(exist_ok=True)
    joblib.dump(masks, out_dir / f"auto_masks_{run_id}.pkl")

    print("Running automatic segmentation for volume")
    predictor = build_sam2_tomogram_predictor(model_cfg, sam2_checkpoint, device=device)

    inference_state = predictor.init_state(vol_normed)
    predictor.reset_state(inference_state)

    ann_frame_idx = zSlice

    for ii in range(len(masks)):
        # sam_points = (auto_points[ii, :] * scale).reshape(1, 2)
        ann_obj_id = ii + 1

        _, out_obj_ids, out_mask_logits = predictor.add_new_mask(
            inference_state=inference_state,
            frame_idx=ann_frame_idx,
            obj_id=ann_obj_id,
            mask=masks[ii]["segmentation"],
        )

    video_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(
        inference_state, start_frame_idx=90, reverse=False
    ):
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

        # Pull out Masks for Multiple Classes
    nMasks = len(video_segments[90])
    vol_mask = np.zeros([vol.shape[0], vol.shape[1], vol.shape[2]], dtype=np.uint8)
    for out_frame_idx in list(video_segments):
        for jj in range(nMasks):
            resized_mask = skimage.transform.resize(
                video_segments[out_frame_idx][jj + 1][0,],
                (vol.shape[1], vol.shape[2]),
                anti_aliasing=False,
            )
            mask_update = resized_mask > 0
            vol_mask[out_frame_idx, :, :][mask_update] = jj

    # run propagation throughout the video and collect the results in a dict
    video_segments = {}  # video_segments contains the per-frame segmentation results
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(
        inference_state, start_frame_idx=90, reverse=True
    ):
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

    # Pull out Masks for Multiple Classes
    for out_frame_idx in list(video_segments):
        for jj in range(nMasks):
            resized_mask = skimage.transform.resize(
                video_segments[out_frame_idx][jj + 1][0,],
                (vol.shape[1], vol.shape[2]),
                anti_aliasing=False,
            )
            mask_update = resized_mask > 0
            vol_mask[out_frame_idx, :, :][mask_update] = jj

    # Save the mask and the volume
    print("Saving the masks and the volume")

    joblib.dump(vol_mask, out_dir / f"vol_mask_{run_id}.pkl")

In [5]:
import copick

In [6]:
copick_config = (
    "/hpc/projects/group.czii/krios1.processing/copick/24jul29c/run002/copick_config.json"
)

# Open Copick Project and Query All Available Runs
root = copick.from_file(copick_config)
run_ids = [run.name for run in root.runs]

In [7]:
n_run_ids = len(run_ids)
n_procs = 2
for _iz in range(0, n_run_ids, n_procs):
    processes = []
    for _in in range(n_procs):
        run_id = run_ids[_iz * n_procs + _in]
        p = mp.Process(
            target=run_in_separate_process,
            args=(run_id, _in),
        )
        processes.append(p)
    for p in processes:
        p.start()

    for p in processes:
        p.join()

    for p in processes:
        p.close()

/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/sam2_image_predictor.py:431: UserWarning: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  masks = self._transforms.postprocess_masks(
/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/sam2_image_predictor.py:431: UserWarning: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously report

Using device:  1
using device: cuda
Getting denoised Tomogram with 10 A voxel size for the associated runID: Position_10_3_Vol
Preprocessing the volume for Position_10_3_Vol
Running automatic segmentation for projection
Saving teh masks
Running automatic segmentation for volume


 40%|███▉      | 74/186 [00:02<00:04, 26.70it/s]

Using device:  0
using device: cuda
Getting denoised Tomogram with 10 A voxel size for the associated runID: Position_10_2_Vol
Preprocessing the volume for Position_10_2_Vol
Running automatic segmentation for projection
Saving teh masks
Running automatic segmentation for volume


 74%|███████▍  | 138/186 [00:06<00:02, 23.41it/s]Process Process-2:
Traceback (most recent call last):
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1757791/372138728.py", line 124, in run_in_separate_process
  File "/hpc/mydata/saugat.kandel/sam2_projects/.pixi/envs/default/lib/python3.12/site-packages/torch/utils/_contextlib.py", line 36, in generator_context
    response = gen.send(None)
               ^^^^^^^^^^^^^^
  File "/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/sam2_tomogram_predictor.py", line 718, in propagate_in_video
    raise RuntimeError("No points are provided; please add points first")
RuntimeError: No points are provided; please a

...

/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/sam2_image_predictor.py:431: UserWarning: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  masks = self._transforms.postprocess_masks(
/hpc/mydata/saugat.kandel/sam2_projects/segment-anything-2/sam2/sam2_image_predictor.py:431: UserWarning: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously report

Using device:  0
using device: cuda
Getting denoised Tomogram with 10 A voxel size for the associated runID: Position_1_3_Vol
Preprocessing the volume for Position_1_3_Vol
Running automatic segmentation for projection
Saving teh masks
Running automatic segmentation for volume


  2%|▏         | 3/186 [00:00<00:06, 28.27it/s]]

Using device:  1
using device: cuda
Getting denoised Tomogram with 10 A voxel size for the associated runID: Position_1_4_Vol
Preprocessing the volume for Position_1_4_Vol
Running automatic segmentation for projection
Saving teh masks
Running automatic segmentation for volume


 63%|██████▎   | 118/186 [00:04<00:02, 31.21it/s]